In [1]:
import lumapi
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
N_CORE   = np.sqrt(12.0)
N_BG     = 1.0
LAMBDA0  = 1.55e-6
DX       = 20e-9

GRID_X_LENGTH = 7.0e-6
GRID_Y_LENGTH = 12e-6
GRID_Z_LENGTH = 1.5e-6   

BUS_LENGTH   = 2.0e-6
BUS_WIDTH    = 0.2e-6
BUS_CENTER_X = 1.0e-6
BUS_CENTER_Y = 0

WEDGE_ARM_LENGTH_X  = 0.5e-6
WEDGE_ARM_OFFSET_Y  = 4.9e-6
WEDGE_RECT_LENGTH_X = 0.5e-6

DESIGN_LENGTH_X      = 1.0e-6
DESIGN_FILL_FRACTION = 0.5

NUM_OUTPUT_WG         = 12
OUTPUT_WG_LENGTH_X    = 2.0e-6
OUTPUT_WG_WIDTH       = 0.2e-6
OUTPUT_WG_GAP         = 0.64e-6
OUTPUT_WG_CENTER_SPACING = OUTPUT_WG_WIDTH + OUTPUT_WG_GAP


DESIGN_LENGTH_X = 1.0e-6
DESIGN_SPAN_Y   = 10.0e-6     

DUMMY_LENGTH_Z = 0.5e-6    

In [ ]:
fdtd = lumapi.FDTD()
fdtd.newproject()
fdtd.addfdtd()


# ========================
# Set up FDTD region
# ========================
fdtd.set("dimension", "3D")
fdtd.set("x span", GRID_X_LENGTH)
fdtd.set("y span", GRID_Y_LENGTH)
fdtd.set("z span", GRID_Z_LENGTH) 

fdtd.set("x", GRID_X_LENGTH/2)
fdtd.set("y", 0.0)
fdtd.set("z", 0.0)
fdtd.set("index", N_BG)

fdtd.set("x min bc", "PML")
fdtd.set("x max bc", "PML")
fdtd.set("y min bc", "PML")
fdtd.set("y max bc", "PML")
fdtd.set("z min bc", "PML")
fdtd.set("z max bc", "PML")

fdtd.addmesh()
fdtd.set("name", "mesh_global")

fdtd.set("x", GRID_X_LENGTH/2)
fdtd.set("y", 0.0)
fdtd.set("z", 0.0)
fdtd.set("x span", GRID_X_LENGTH)
fdtd.set("y span", GRID_Y_LENGTH)
fdtd.set("z span", GRID_Z_LENGTH)

fdtd.set("dx", DX)
fdtd.set("dy", DX)
fdtd.set("dz", DX)

# -----------------------
# 1) Input bus waveguide (rect)
# -----------------------
fdtd.addrect()
fdtd.set("name", "bus")
fdtd.set("x", BUS_CENTER_X)
fdtd.set("y", BUS_CENTER_Y)
fdtd.set("z", 0.0)
fdtd.set("x span", BUS_LENGTH)
fdtd.set("y span", BUS_WIDTH)
fdtd.set("z span", DUMMY_LENGTH_Z)   
fdtd.set("index", N_CORE)

bus_right_edge_x = BUS_CENTER_X + BUS_LENGTH/2



# -----------------------
# 2) Wedge / taper (polygon)
# -----------------------
taper_length_x = WEDGE_ARM_LENGTH_X + WEDGE_RECT_LENGTH_X
taper_x0 = bus_right_edge_x
taper_x1 = bus_right_edge_x + taper_length_x

w0 = BUS_WIDTH
w1 = DESIGN_SPAN_Y
y0 = BUS_CENTER_Y


fdtd.addpoly()
fdtd.set("name", "wedge_taper")
fdtd.set("x", 0)  
fdtd.set("y", 0)  
fdtd.set("z", 0.0)
fdtd.set("z span", DUMMY_LENGTH_Z)   
fdtd.set("index", N_CORE) 
verts = np.array([
    [taper_x0, y0 - w0/2],
    [taper_x0, y0 + w0/2],
    [taper_x1, y0 + w1/2],
    [taper_x1, y0 - w1/2],
])
fdtd.set("vertices", verts)


rect_length_x = WEDGE_RECT_LENGTH_X  # 0.5e-6
rect_x0 = taper_x1
rect_center_x = rect_x0 + rect_length_x/2

fdtd.addrect()
fdtd.set("name", "rectangle1")
fdtd.set("x", rect_center_x)
fdtd.set("y", y0)
fdtd.set("z", 0.0)
fdtd.set("x span", rect_length_x)
fdtd.set("y span", DESIGN_SPAN_Y)
fdtd.set("z span", DUMMY_LENGTH_Z)   
fdtd.set("index", N_CORE)


# # -----------------------
# # 3) Solid design region 
# # -----------------------

design_left_edge_x = rect_x0 + rect_length_x
design_right_edge_x = design_left_edge_x + DESIGN_LENGTH_X
design_center_x = 0.5*(design_left_edge_x + design_right_edge_x)

design_center_y = 0.0
design_span_x   = DESIGN_LENGTH_X
design_span_y   = DESIGN_SPAN_Y
design_span_z = DUMMY_LENGTH_Z  # your desired thickness

fdtd.addrect()
fdtd.set("name", "design_spatial")
fdtd.set("x", design_center_x)
fdtd.set("y", y0)
fdtd.set("z", 0.0)
fdtd.set("x span", design_span_x)
fdtd.set("y span", design_span_y)
fdtd.set("z span", DUMMY_LENGTH_Z)   # match fdtdx
fdtd.set("index", N_CORE)



# -----------------------
# 4) Output waveguides
# -----------------------
output_left_edge_x = design_right_edge_x
output_center_x    = output_left_edge_x + OUTPUT_WG_LENGTH_X/2

mid = (NUM_OUTPUT_WG - 1) / 2.0
for i in range(NUM_OUTPUT_WG):
    y_c = (i - mid) * OUTPUT_WG_CENTER_SPACING  # centered around y=0

    fdtd.addrect()
    fdtd.set("name", f"out_wg_{i:02d}")
    fdtd.set("x", output_center_x)
    fdtd.set("y", y_c)
    fdtd.set("z", 0.0)
    fdtd.set("x span", OUTPUT_WG_LENGTH_X)
    fdtd.set("y span", OUTPUT_WG_WIDTH)
    fdtd.set("z span", DUMMY_LENGTH_Z)
    fdtd.set("index", N_CORE)


# ---------- Source in bus ----------
LAMBDA_C = 1.55e-6
LAMBDA_SPAN = 1e-5  # covers 1.54–1.56 µm

bus_left_edge_x = BUS_CENTER_X - BUS_LENGTH/2
x_src = bus_left_edge_x + 0.1e-6

fdtd.addmode()
fdtd.set("name", "src_bus")
fdtd.set("injection axis", "x-axis")
fdtd.set("direction", "Forward")
fdtd.set("x", x_src)
fdtd.set("y", BUS_CENTER_Y)
fdtd.set("z", 0.0)
fdtd.set("y span", 0.4e-6)
fdtd.set("z span", 1e-6)

# Gaussian spectrum centered at 1.55 µm
fdtd.set("center wavelength", LAMBDA_C)
fdtd.set("wavelength span", LAMBDA_SPAN)

# -----------------------
# Power monitors at output waveguide ends
# -----------------------
output_left_edge_x   = design_right_edge_x
output_center_x      = output_left_edge_x + OUTPUT_WG_LENGTH_X/2
output_right_edge_x  = output_center_x + OUTPUT_WG_LENGTH_X/2

x_mon = output_right_edge_x 
mid = (NUM_OUTPUT_WG - 1) / 2.0

for i in range(NUM_OUTPUT_WG):
    y_c = BUS_CENTER_Y + (i - mid) * OUTPUT_WG_CENTER_SPACING

    fdtd.adddftmonitor()
    fdtd.set("name", f"mon_out_{i:02d}")
    fdtd.set("monitor type", "2D X-normal")
    fdtd.set("x", x_mon)
    fdtd.set("y", y_c)
    fdtd.set("z", 0.0)
    fdtd.set("y span", OUTPUT_WG_WIDTH)
    fdtd.set("z span", DUMMY_LENGTH_Z)

